<a href="https://colab.research.google.com/github/pyo9912/AI_Study/blob/master/PEFT/mt0-large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PEFT LoRa와 Accelerate의 deepspeed를 사용하여 bioscience/t0_3B모델을 파인튜닝하기

In [ ]:
pip install transformers

In [ ]:
pip install peft

### 1. 필수 모듈 import

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_model, LoraConfig, TaskType

### 2. PEFT config 만들기  
- 각 PEFT method는 PeftConfig에 의해 정의되며 이는 PeftModel을 만들기 위해 중요한 파라미터를 모두 담고 있음
- 여기서는 LoRA 방법론을 이용하고자 하기 때문에 LoRaConfig class를 생성하며 다음의 파라미터를 정해줌 (adapters, LoRA, prompt tuning, prefix tuning 등의 방법론 존재)
  - task_type: 여기서는 seq-to-seq LM
  - inference_mode: 모델을 inference에 사용할 것인지 여부
  - r: low-rank matrices의 dimension
  - lora_alpha: low-rank matrices의 scaling factor
  - lora_dropout: LoRA layer의 dropout probability

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

### 3. get_peft_model 함수 안에 허깅페이스 모델과 설정해둔 config 넣어주기
- 본 실습에서는 LoRa 방법을 이용해서 허깅 페이스가 제공하는 pre-trained 모델인 bigscience/mt0-large를 파인튜닝
- PeftModel은 **get_peft_model()** 함수로 생성되며 Transformer library에서 load할 수 있는 base-model을 가져옴
- PeftConfig는 PEFT method를 위한 모델 구성을 담고 있음
- print.trainable_parameters()를 통해 모델의 파라미터 중 학습 가능한 파라미터의 비율을 확인

In [ ]:
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.19151053100118282


### 4. 모델 저장
- 모델 학습이 끝난 후 모델을 디렉토리에 저장
- **save_prtrained()** 함수를 이용
- 학습된 PEFT weights의 증분을 저장하기 때문에 store, transfer, load에 굉장히 효율적임

In [ ]:
model.save_pretrained("output_dir")

### 5. 모델 불러오기 및 모델 이용

tokenizer issue 해결

In [ ]:
!pip install sentencepiece

In [40]:
pip uninstall transformers

Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.31.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.31.0


In [41]:
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 84.9 MB/s eta 0:00:00


In [7]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from peft import PeftModel, PeftConfig

import torch
from torch import nn

peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, use_fast=False)

device = torch.device("cuda")

model = model.to(device)
model.eval()

tweet_text = "@HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again."
inputs = tokenizer(tweet_text, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)

with torch.no_grad():
    outputs = model.generate(input_ids=input_ids, max_length=inputs["input_ids"].shape[-1] + 10)
    decoded_outputs = tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)[0]
    print("Generated text:", decoded_outputs)

Generated text: complaint
